In [1]:
%config InlineBackend.figure_formats = ['svg']
import cvxpy as cp
import numpy as np
import pandas as pd; pd.set_option('display.max_rows', 200)
import geopandas as gpd
import matplotlib.pyplot as plt
from ed_institutions import load_ed_institutions

In [2]:
# Tufts has ~90% of beds left right now (without accounting for inaccessible dorms?)
# [Unknown] Is ~80% capacity over all universities a reasonable estimate?
ed_inst_capacity = 0.8
# This parameterization gives us 35 universities.
ed_inst_min_beds = 500
ed_inst_min_endowment = 50000000
# Per Mike Apkon: each patient receives 12 person-hours of care per day (under normal conditions).
# Assuming a 40-hour workweek, this implies 1.5 staff members are fully allocated to each patient on each day.
# (Note: does this average include ICU patients? Presumably they get more care.)
# General consensus seems to be that care per patient will *decrease* as hospitals get overloaded with COVID cases,
# so consider staffing estimates to be an upper bound.
staff_hours_per_day = 8
staff_days_per_week = 5
patient_person_hours_per_day = 12
staff_per_patient = patient_person_hours_per_day / staff_hours_per_day
# [Unknown] What percentage of non-COVID patients can be moved into a dorm?
# (This parameter also encapsulates the fact that most patients probably need to present at the hospital
#  and *then* moved to the dorms.)
patient_bed_demand_pct = 0.5
# [Unknown] What percentage of staff will want to stay in a dorm rather than going home?
staff_bed_demand_pct = 0.5
# [Unknown] We assume that staff is assigned to 100% of hospital+ICU beds at the peak of the outbreak.
staff_utilized_pct = 1

# TODO: not entirely sure of the relevance of these fudge factors -- 
#       shouldn't we just be able to get discharge rates empirically?
community_discount = 0.5  # Discount community hospital normal occupancy by 50%
academic_discount = 0.75  # Discount academic hospital normal occupancy by 75%

# TODO: choose some reasonable parameters here! may need to do some sensitivity analysis
staff_transport_cost_per_min = 1
patient_transport_cost_per_min = 100

# TODO: max distance cutoffs not implemented yet
max_dist_min = 60

crisis_length = 14  # in days
staff_commutes = 2 * (crisis_length / staff_days_per_week)

SyntaxError: invalid syntax (<ipython-input-2-68abd1fc4183>, line 6)

In [ ]:
# Ballpark parameters (to be replaced when more data comes in)
avg_stay_days = 5
patient_turnover = crisis_length / avg_stay_days
# Normally, ~70% of beds are utilized in MA.
normal_beds_utilized_pct = 0.7 

## TODO
* Geodata: where are the hospitals? (first pass: don't differentiate b/t acute and non-acute)
* Geodata: where are the __filtered__ institutions? (join w/ ed. institutions spreadsheet
* Plot: Hospitals, universities on MA HRRs (bubbles scaled for capacity)
* Data: Hospital discharge rates (to calculate turnover) [can always just fake this and assume 5-day turnover or something]

In [ ]:
proj = "+proj=utm +zone=18N +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
acute_care_gdf = gpd.read_file('data/Hospitals/Originals/acute_care_hospitals/HOSPITALS_PT.shp').to_crs(proj)
non_acute_care_gdf = gpd.read_file('data/Hospitals/Originals/non_acute_care_hospitals/HOSPITALS_NONACUTE_PT.shp').to_crs(proj)
state_outlines_gdf = gpd.read_file('data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp').to_crs(proj)
ed_inst_gdf = gpd.read_file('data/Universities/SHP_dormcap/ma_universities.shp').to_crs(proj)
ma_outline_gdf = state_outlines_gdf[state_outlines_gdf['NAME'] == 'Massachusetts']

In [ ]:
ed_inst_df = load_ed_institutions('data/ma_dorm_capacity.csv', ed_inst_min_endowment, 0)
ed_inst_gdf = ed_inst_gdf[ed_inst_gdf.COLLEGE.isin(ed_inst_df.name)]
ed_inst_gdf = ed_inst_gdf[ed_inst_gdf['DORMCAP'] >= ed_inst_min_beds].reset_index()

In [ ]:
acute_care_gdf.columns

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
base = ma_outline_gdf.plot(ax=ax, color='#e0e0e0')
acute_care_gdf.plot(ax=base, marker='s', color='red', alpha=0.5,
                    markersize=0.1 * acute_care_gdf['BEDCOUNT'])
non_acute_care_gdf.plot(ax=base, marker='D', color='green', alpha=0.5,
                        markersize=0.1 * non_acute_care_gdf['BEDCOUNT'])
ed_inst_gdf.plot(ax=base, marker='o', color='blue', alpha=0.5,
                 markersize=0.1 * ed_inst_gdf['DORMCAP'])
plt.axis('off')
plt.savefig('ma_beds.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# We treat acute care hospitals and non-acute care hospitals similarly. 
acute_care_gdf['TYPE'] = 'acute'
non_acute_care_gdf['TYPE'] = 'non-acute'
hospitals_gdf = gpd.GeoDataFrame(pd.concat([
    acute_care_gdf,
    non_acute_care_gdf.rename(columns={'FAC_NAME': 'SHORTNAME'})
])).drop(columns=set(acute_care_gdf.columns) - set(non_acute_care_gdf.columns)) \
.rename(columns={'SHORTNAME': 'NAME'})

# Remove island hospitals from consideration (not accessible by road)
hospitals_gdf = hospitals_gdf[(hospitals_gdf['TOWN'] != 'Oak Bluffs') & (hospitals_gdf['TOWN'] != 'Nantucket')]
hospitals_gdf = hospitals_gdf.reset_index()

# TODO: Remove hospitals not within X miles/minutes of an institution?

In [ ]:
n_hosp = len(hospitals_gdf)
n_ed = len(ed_inst_gdf)

In [ ]:
# TODO: insert Olivia's road distances here!
distances = np.zeros((n_hosp, n_ed))
for hosp_idx, hosp_row in hospitals_gdf.iterrows():
    for ed_idx, ed_row in ed_inst_gdf.iterrows():
        dist = hosp_row.geometry.distance(ed_row.geometry)
        distances[hosp_idx, ed_idx] = dist

In [ ]:
dorm_bed_capacity = np.round(ed_inst_capacity * ed_inst_gdf['DORMCAP'].to_numpy())
patient_bed_demand = np.round(normal_beds_utilized_pct * patient_bed_demand_pct * hospitals_gdf['BEDCOUNT'].to_numpy())
staff_bed_demand = np.round(staff_per_patient * staff_bed_demand_pct * staff_utilized_pct * hospitals_gdf['BEDCOUNT'].to_numpy())

In [ ]:
staff_assignment = cp.Variable((n_hosp, n_ed), integer=True)
patient_assignment = cp.Variable((n_hosp, n_ed), integer=True)

In [ ]:
constraints = [
    staff_assignment >= 0,
    patient_assignment >= 0,
    cp.sum(staff_assignment, axis=0) + cp.sum(patient_assignment, axis=0) <= dorm_bed_capacity,
    cp.sum(staff_assignment, axis=1) == staff_bed_demand,
    cp.sum(patient_assignment, axis=1) == patient_bed_demand,
]
objective = cp.Minimize(cp.sum(
    cp.multiply(distances, (
        (staff_commutes * staff_transport_cost_per_min * staff_assignment) + 
        (patient_turnover * patient_transport_cost_per_min * patient_assignment)
    ))
))
prob = cp.Problem(objective, constraints=constraints)

In [ ]:
prob.solve()

In [ ]:
staff_results = np.round(staff_assignment.value)
patient_results = np.round(patient_assignment.value)

In [ ]:
def plot_assignments(results):
    fig, ax = plt.subplots(figsize=(16, 8))
    base = ma_outline_gdf.plot(ax=ax, color='#e0e0e0')
    hospitals_gdf.plot(ax=base, marker='s', color='red', alpha=0.5,
                        markersize=0.1 * hospitals_gdf['BEDCOUNT'])
    ed_inst_gdf.plot(ax=base, marker='o', color='blue', alpha=0.5,
                     markersize=0.1 * ed_inst_gdf['DORMCAP'])

    for hosp_idx in range(n_hosp):
        for ed_idx in range(n_ed):
            if results[hosp_idx, ed_idx] > 0:
                p_hosp = hospitals_gdf.iloc[hosp_idx].geometry
                p_ed = ed_inst_gdf.iloc[ed_idx].geometry
                ax.plot([p_hosp.x, p_ed.x], [p_hosp.y, p_ed.y],
                        color='black',
                        linewidth=0.01 * results[hosp_idx, ed_idx])
                
    plt.axis('off')

In [ ]:
plot_assignments(staff_results)
plt.title('Staff assignments')
plt.savefig('ma_staff_assignments.png', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
plot_assignments(patient_results)
plt.title('Patient assignments')
plt.savefig('ma_patient_assignments.png', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
staff_by_inst = np.sum(staff_results, axis=0)
patients_by_inst = np.sum(patient_results, axis=0)
ed_assignment_df = ed_inst_gdf[['COLLEGE', 'CAMPUS', 'CITY', 'ENDOW', 'DORMCAP']].copy()
ed_assignment_df['staff_assigned'] = staff_by_inst
ed_assignment_df['patients_assigned'] = patients_by_inst
ed_assignment_df['utilization'] = (ed_assignment_df['staff_assigned'] + ed_assignment_df['patients_assigned']) / ed_assignment_df['DORMCAP']
ed_assignment_df.to_csv('ed_inst_assignments.csv', index=False)